# Python Classes for Vector Calculus

This notebook overviews a Python library based on "Object structure of vector calculus", hereinafter *VC*. The central thrust of VC is that a mimetic approach to vector calculus, with computation imitating mathematics as closely as possible, is necessarily object-oriented. The essential objects are also structurally constrained.

The core mathematical types of vector calculus are normed vector spaces, vectors, linear operators or maps, and differentiable functions. My particular objective is mimetic expression of continuous optimization algorithms, which generally presume an inner product, so norms are assumed here to be inner product norms. This notebook reviews a Python realization of the structure explained in VC, with a few simple examples based on NumPy. 

## Spaces and Vectors

As explained in VC, (pre-Hilbert) vector spaces combine a set of data objects with two operations on them, linear combination and inner (dot) product. The set of data objects has infinite cardinality except in one obvious instance, so is not computationally realizable. However it is sufficient to be able to determine whether an object is a member of the set, and to obtain a new object in the set on request ("let $v \in V$"). These observations translate into four pseudo-code attributes of a Space object:
1. a boolean function that takes an object argument and returns True if the argument refers to a data object for this space;
2. a function with no arguments that returns a new data object;
3. a linear combination function that evaluates $y=ax + by$, with arguments $a$, $b$ (scalars), $x$, and $y$ (data objects). Error if either $x$ or $y$ is not a valid data object;
4. an inner product function that returns a scalar $\langle x, y \rangle$ for arguments $x$ and $y$. Error if either $x$ or $y$ is not a data object.

All Spaces have these attributes, so the collection of Spaces is naturally expressed as an abstract class. In Python,

In [1]:
from abc import ABC, abstractmethod

class Space(ABC):
    @abstractmethod
    def isData(self,x):
        pass
    @abstractmethod
    def getData(self):
        pass
    @abstractmethod
    def linComb(self, a, x, y, b=1.0):
        pass
    @abstractmethod    
    def dot(self,x,y):
        pass
    ...

The *Space* class declaration in *vcl.py* includes several other convenient methods, including a self-description interface and a *cleanup* method to remove any part of a data object that Python garbage collection does not automatically remove.

While vector space is an abstract concept, emulated in code by an abstract type, the definition of vector is entirely concrete: it is a data object (a member of $V$, in the notation of the paper) *together with* with a vector space and its attributes. The almost-universal mathematical vernacular calls the data objects "vectors" As explained in VC, this is logically incorrect, but also misleading: data objects must be combined with the other attributes of their vector space to act functionally as vectors. So a vector is not a data object alone, but a composite of a data object *together with* a vector space with its linear combination and inner product functions, for which the data object belongs to its proper set of data objects.

While *vcl.Space* is an abstract type, asserting behaviour but not implementing it, every aspect of vector behaviour is determined by attributes of the corresponding space. So the Python realization *vcl.Vector* is a concrete, rather than abstract, class - its attributes are defined, not merely declared:


In [2]:
class Vector:

    def __init__(self, sp):
        self.space = sp
        self.data = sp.getData()

    def __del__(self):
        self.space.cleanup(self.data)
            
    def linComb(self,a,x,b=1.0):
        self.space.linComb(a,x.data,self.data,b)

    def dot(self,x):
        return self.space.dot(self.data,x.data)
    ...

The full definition in *vcl.py* includes a mechanism for assigning a *Vector* to an existing data object, rather than the new data object assigned to it on construction. This possibility is convenient in applications. Several other convenience attributes are also provided.

For a simple example, I will construct a vector space based on NumPy. This choice makes a point: NumPy is already a fine environment for matrix algebra. However it does not offer interfaces for functions on subsets of vector spaces, nor for expression of algorithms defined in terms of vector functions, such as Newton's method. NumPy's array manipulation capabilities support straightforward construction of a vector space type, in the form of a *Space* as defined (partly) above. The key choice is to us NumPy *ndarrays* as the data objects. Each space corresponds mathematically to ${\bf R}^n$, so is characterized by its dimension. So an object is a data object of an *npSpace* if it is an column *numpy.ndarray* of the right dimension. 

In [3]:
import vcl
import numpy as np

# numpy vector space
class npSpace(vcl.Space):
    def __init__(self,n):
        self.dim = n
    def getData(self):
        return np.zeros(self.dim).reshape(self.dim,1)
    def isData(self,x):
        return (isinstance(x,np.ndarray) and x.shape() == (self.dim,1))
    def linComb(self,a,x,y,b=1.0):
        y = a*x + b*y
    def dot(self,x,y):
        return np.dot(x.T,y)[0][0]

*npSpace* is a sub- (or derived) class of *Space*, as indicated by the first line in the definition. So it can be used in any context that calls for a *Space*.

The full definition in *npvc.py* includes several other useful functions, that are (or could be) defined in terms of the core functions described above. The *lincomb* and *dot* functions (along with the others) are also written to provide error messages on failure.

Here is a very simple use of the *npSpace* and *Vector* classes:

In [11]:
import numpy as np
import vcl
import npvc

dom = npvc.Space(2)
x=vcl.Vector(dom)
x.data[0]=1
x.data[1]=1
print('A vector in 2D NumPy-based Space:')
x.myNameIs()

A vector in 2D NumPy-based Space:
Vector in space:
npvc.Space of dimension 2
Data object:
numpy data:
[[1.]
 [1.]]


This example, simple as it is, makes another important point. There are NumPy *Space*s (namely *npSpace*s), but there is no NumPy *Vector*: there are only *Vector*s in *npSpace*s. The vector concept does not need to be specialized: it is a "function" of the choice of space, and is otherwise completely defined.

## Linear Operators

The third fundamental concept of linear algebra, after vector space and vector, is that of linear map or operator. These are simply functions whose domains and ranges are vector spaces, and which satisfy the linearity condition. Since the vector spaces at issue are presumed to be inner product spaces, linear operators really come in adjoint pairs. A suitable Python abstract class defining the behaviour of linear operators is

In [5]:
class LinearOperator:
    @abstractmethod
    def getDomain(self):
        pass
    @abstractmethod
    def getRange(self):
        pass
    @abstractmethod
    def applyFwd(self,x, y):
        pass
    @abstractmethod
    def applyAdj(self,x, y):
        pass        
    @abstractmethod
    def myNameIs(self):
        pass

Besides being a logical imperative, enabling the *LinearOperator* instance to return references to its domain and range is actually useful in the formulation of algorithms, as will be seen below.

The obvious linear operator class for *npSpace* performs matrix-vector multiplication:

In [6]:
class MatrixOperator(vcl.LinearOperator):    
    def __init__(self,dom,rng,mat):
        self.dom = dom
        self.rng = rng
        self.mat = np.copy(mat)
        #....    
    def applyFwd(self,x, y):
        y.data = self.mat@x.data
    def applyAdj(self,x, y):
        y.data = self.mat.T@x.data

The elided code in the constructor checks that the *npSpaces* *dom* and *rng* have the number of columns of the *numpy.ndarray*, respectively its number of rows, as their dimensions. 

Note that the domain and range *Space*s are passed as arguments, therefore stored as references to external objects that exist independently of the *MatrixOperator*, whereas the matrix argument is copied, internal to the object. This construction requires the error-checking just mentioned, but has several advantages. Obviously the domain and range spaces could also be copied, but that would be logically incorrect as well as inconvenient. With the *MatrixOperator* storing references to externally defined spaces, membership in those spaces can be verified by simple comparison: to know that a *vcl.Vector* x is a member of the domain, for example, simply check the value of *x.space==self.dom*. It is not enough to check that the input vector and the domain have the same dimension. For example, subclasses of *npvc.Space* could be equipped with units, whence only checking equality of dimensions could lead to dimensional errors. Insisting that the spaces involved should be *the same objects* avoids such egregious errors.

The matrix, on the other hand, is naturally internal data. This construction maintains the identity of the *MatrixOperator* even if the NumPy array passed to the constructor is subsequently changed.

Here is a simple example of matrix multiplication via the *npvc.MatrixOperator* class. The textual overhead from expressing matrix multiplication as the action of a linear operator (as compared to straight NumPy) is: 3 lines of code, out of 15.

In [7]:
import numpy as np
import vcl
import npvc

# domain space and vector in it
dom = npvc.Space(2)
x=vcl.Vector(dom)
x.data[0]=1
x.data[1]=1

# range space and vector in it
rng = npvc.Space(3)
y=vcl.Vector(rng)

# 3 x 2 matrix - initialize as outer product
#mat=y.data@x.data.T
mat=np.zeros((3,2))
mat[0,0]=1
mat[0,1]=1
mat[1,1]=1

# matrix operator based on mat
matop=npvc.MatrixOperator(dom,rng,mat)

# matrix-vector product as matrix operator 
# application
matop.applyFwd(x,y)

print('Input vector x')
x.myNameIs()
print('Matrix Operator matop')
matop.myNameIs()
print('Output vector y = matop(x)')
y.myNameIs()

Input vector x
Vector in space:
npvc.Space of dimension 2
Data object:
numpy data:
[[1.]
 [1.]]
Matrix Operator matop
NUMPY Matrix Operator with matrix:
[[1. 1.]
 [0. 1.]
 [0. 0.]]
domain:
npvc.Space of dimension 2
range:
npvc.Space of dimension 3
Output vector y = matop(x)
Vector in space:
npvc.Space of dimension 3
Data object:
numpy data:
[[2.]
 [1.]
 [0.]]


# Functions

Having constructed a class for linear operators (i.e. linear functions), it's obvious how to build a class for (possibly) non-linear functions. First, absent linearity there is no natural adjoint concept, so there is only a "forward" application function. Second, for differentiable functions a derivative (a linear operator) is another attribute. 

These considerations suggest a very simple abstract interface:

In [8]:
class Function(ABC):
    @abstractmethod
    def getDomain(self):
        pass
    @abstractmethod
    def getRange(self):
        pass
    @abstractmethod
    def apply(self,x,y):
        pass
    # should return linear op
    @abstractmethod
    def deriv(self,x):
        pass
    @abstractmethod
    def myNameIs(self):
        pass

A simple NumPy-based example is provided as *npvc.OpExpl1*. It realizes the function $f: {\bf R}^2 \rightarrow {\bf R}^3$ given by 
$$
f((x_0,x_1)^T) = (x_0*x_1, -x_1+x_0^2, x_1^2)^T.
$$
Its code is written according the principles outlined above. For instance, the domain and range spaces are constructed externally to the function object, and passed to its constructor as arguments. Their dimensions are checked to be 2 and 3 respectively. The function object stores references to these externally defined spaces. The *apply* and *deriv* methods sanity-check their arguments. See the code for details.

In [9]:
dom = npvc.Space(2)
rng = npvc.Space(3)
f = npvc.OpExpl1(dom,rng)
x = vcl.Vector(dom)
y = vcl.Vector(rng)
x.data[0]=1
x.data[1]=-2
print('input vector:')
x.myNameIs()
f.apply(x,y)
print('output of apply method:')
y.myNameIs()
dfx = f.deriv(x)
print('output of deriv method:')
dfx.myNameIs()

input vector:
Vector in space:
npvc.Space of dimension 2
Data object:
numpy data:
[[ 1.]
 [-2.]]
output of apply method:
Vector in space:
npvc.Space of dimension 3
Data object:
numpy data:
[[-2.]
 [ 3.]
 [ 4.]]
output of deriv method:
NUMPY Matrix Operator with matrix:
[[-2.  1.]
 [ 2. -1.]
 [ 0. -4.]]
domain:
npvc.Space of dimension 2
range:
npvc.Space of dimension 3


Of course linear functions (operators) are also functions, so really *LinearOperator* should subclass *Function*. The derivative is a linear operator, so the definition of *Function* might seem to depend of the definition of *LinearOperator* which in turn depends on *Function*. However it is possible to take advanage of Python's genericity to break this cyclic dependence: the return type of the *Function.deriv* method isn't specified, because return types are not specified in Python! So re-write the definition of *LinearOperator* as follows:

In [12]:
class LinearOperator(Function):
    def apply(self,x, y):
        self.applyFwd(x,y)
    def deriv(self,x):
        return self
    @abstractmethod
    def applyFwd(self,x, y):
        pass
    @abstractmethod
    def applyAdj(self,x, y):
        pass 

That is, *LinearOperator* is *Function* with two abstract methods made concrete (defined), two abstract methods added, and the others inherited. Of course the *deriv* method returns *self*, since a linear operator is its own derivative.

## Jets

The *jet* concept solves an obvious consistency vs. efficiency problem involving the attributes of functions. Suppose that *f* is a *vcl,Function*, and *x* is a *vcl.Vector*. Record the function's value in the *vcl.Vector* *y*, and the derivative in the *vcl.LinearOperator* *df*. Some number of lines later in the program, access these same objects. Is there any guarantee that they are still related in the same way? In fact, there is not. If *x* is changed but the function is not re-evaluated, then the three objects have become inconsistent. The only way to make sure that this inconsistency does not occur appears to be re-computing the value and derivative each time they are accessed. That could amount to considerable wasted computational effort.

Jets, in the sense meant here, borrow a concept from differential geometry, and offer a way around this problem. There are several equivalent definitions, of which I cite the one most relevant to computation. The $k$-jet of a $C^{\infty}$ function $f$ at a point $x$ in its domain is the sequence of values $D^{\alpha}f(x)$ of $f$ and all of its derivatives of orders $|\alpha| \le k$. This definition suggests an obvious container class, implemented in $vcl.Jet$. For the time being, I have implemented only the 1-jet. The constructor arguments are the *vcl.Function* *f* and *vcl.Vector* *x*. The methods *getPoint*, *getValue*, and *getDeriv* return the computational analogues of $x$, $f(x)$, and $Df(x)$ respectively. These are stored as internal copies, so changing *x* after creation of *vcl.Jet(f,x)* does not change the return values of these methods. Thus an instance of *vcl.Jet* provides access to a consistent set of values.

An earlier implementation of this concept in the C++ library RVL used access control and the *const* keyword to prevent any violation of the jet's internal data, so really offered a guarantee that the return values of its methods are coherent. Such guarantees are impossible to provide in Python, which does not implement *const* and makes all class data public. So *vcl.Jet* really only provides guidance to help the programmer maintain the coherence of function values. Vector data is also private in RVL, and can only be altered through the action of a few specified function classes. This restriction makes a *versioning* system possible. The jet methods compare the version index of a vector with a recorded index to tell whether the vector had been altered, thus update values and derivatives automatically whenever necessary. So RVL implements the jet concept as *f(x) for variable x*. Such automation is impossible in a Python framework: the VCL user is responsible for updating *vcl.Jet* instances as needed. 

## Scalar Functions

## Product Spaces and Partial Derivatives

## Variable Projection

Variable projection is a minimization algorithm for a function of two (vector) variables $f(x_1,x_2)$, of class $C^2$. Suppose that for each $x_1$, $\tilde{x}_2(x_1)$ is a local minimizer of $x_2 \rightarrow f(x_1,x_2)$, and that $\tilde{x}_2$ is of class $C^1$. This is the case if, for example, the Hessian $D_2^2 f(x_1,\tilde{x}_2 (x_1))$ is positive definite symmetric. Define $\tilde{f}(x_1)=f(x_1,\tilde{x}_2 (x_1))$. It follows directly from the chain rule that if $x_1$ is a stationary point of $\tilde{f}$, then $(x_1,\tilde{x}_2(x_1))$ is a stationary point of $f$.

## Algorithms
